Järgmiste märkmike käivitamiseks, kui te pole seda veel teinud, peate seadistama openai võtme .env faili nimega `OPENAI_API_KEY`


In [ ]:
import os
import pandas as pd
import numpy as np
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()

API_KEY = os.getenv("OPENAI_API_KEY","")
assert API_KEY, "ERROR: OpenAI Key is missing"

client = OpenAI(
    api_key=API_KEY
    )

model = 'text-embedding-ada-002'

SIMILARITIES_RESULTS_THRESHOLD = 0.75
DATASET_NAME = "../embedding_index_3m.json"

Järgmisena laadime manustamisindeksi Pandase andmeraamatusse. Manustamisindeks on salvestatud JSON-faili nimega `embedding_index_3m.json`. Manustamisindeks sisaldab manustusi iga YouTube'i transkriptsiooni jaoks kuni 2023. aasta oktoobri lõpuni.


In [ ]:
def load_dataset(source: str) -> pd.core.frame.DataFrame:
    # Load the video session index
    pd_vectors = pd.read_json(source)
    return pd_vectors.drop(columns=["text"], errors="ignore").fillna("")

Järgmisena loome funktsiooni nimega `get_videos`, mis otsib päringu jaoks Embedding Indexist. Funktsioon tagastab 5 kõige sarnasemat videot päringule. Funktsioon töötab järgmiselt:

1. Esiteks luuakse Embedding Indexist koopia.
2. Seejärel arvutatakse päringu Embedding, kasutades OpenAI Embedding API-d.
3. Seejärel luuakse Embedding Indexisse uus veerg nimega `similarity`. Veerg `similarity` sisaldab kosinussarnasust päringu Embeddingu ja iga videosalvestise Embeddingu vahel.
4. Järgmisena filtreeritakse Embedding Index `similarity` veeru alusel. Embedding Index filtreeritakse nii, et sisaldab ainult videoid, mille kosinussarnasus on suurem või võrdne 0.75-ga.
5. Lõpuks sorteeritakse Embedding Index `similarity` veeru järgi ja tagastatakse 5 parimat videot.


In [ ]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def get_videos(
    query: str, dataset: pd.core.frame.DataFrame, rows: int
) -> pd.core.frame.DataFrame:
    # create a copy of the dataset
    video_vectors = dataset.copy()

    # get the embeddings for the query    
    query_embeddings = client.embeddings.create(input=query, model=model).data[0].embedding

    # create a new column with the calculated similarity for each row
    video_vectors["similarity"] = video_vectors["ada_v2"].apply(
        lambda x: cosine_similarity(np.array(query_embeddings), np.array(x))
    )

    # filter the videos by similarity
    mask = video_vectors["similarity"] >= SIMILARITIES_RESULTS_THRESHOLD
    video_vectors = video_vectors[mask].copy()

    # sort the videos by similarity
    video_vectors = video_vectors.sort_values(by="similarity", ascending=False).head(
        rows
    )

    # return the top rows
    return video_vectors.head(rows)

See funktsioon on väga lihtne, see lihtsalt prindib välja otsingupäringu tulemused.


In [ ]:
def display_results(videos: pd.core.frame.DataFrame, query: str):
    def _gen_yt_url(video_id: str, seconds: int) -> str:
        """convert time in format 00:00:00 to seconds"""
        return f"https://youtu.be/{video_id}?t={seconds}"

    print(f"\nVideos similar to '{query}':")
    for _, row in videos.iterrows():
        youtube_url = _gen_yt_url(row["videoId"], row["seconds"])
        print(f" - {row['title']}")
        print(f"   Summary: {' '.join(row['summary'].split()[:15])}...")
        print(f"   YouTube: {youtube_url}")
        print(f"   Similarity: {row['similarity']}")
        print(f"   Speakers: {row['speaker']}")

1. Esiteks laaditakse manustamise indeks Pandase andmeraamatusse.
2. Seejärel palutakse kasutajal sisestada päring.
3. Seejärel kutsutakse välja funktsioon `get_videos`, et otsida manustamise indeksist päringut.
4. Lõpuks kutsutakse välja funktsioon `display_results`, et kuvada kasutajale tulemused.
5. Seejärel palutakse kasutajal sisestada uus päring. See protsess jätkub, kuni kasutaja sisestab `exit`.

![](../../../../translated_images/et/notebook-search.1e320b9c7fcbb0bc.webp)

Teil palutakse sisestada päring. Sisestage päring ja vajutage enter. Rakendus tagastab nimekirja videodest, mis on päringuga seotud. Rakendus tagastab ka lingi video kohta, kus asub vastus küsimusele.

Siin on mõned päringud, mida proovida:

- Mis on Azure Machine Learning?
- Kuidas töötavad konvolutsioonilised närvivõrgud?
- Mis on närvivõrk?
- Kas ma saan kasutada Jupyter Notebooke koos Azure Machine Learninguga?
- Mis on ONNX?


In [ ]:
pd_vectors = load_dataset(DATASET_NAME)

# get user query from input
while True:
    query = input("Enter a query: ")
    if query == "exit":
        break
    videos = get_videos(query, pd_vectors, 5)
    display_results(videos, query)

---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Vastutusest loobumine**:
See dokument on tõlgitud kasutades tehisintellektil põhinevat tõlketeenust [Co-op Translator](https://github.com/Azure/co-op-translator). Kuigi püüame tagada täpsust, palun arvestage, et automaatsed tõlked võivad sisaldada vigu või ebatäpsusi. Originaaldokument selle emakeeles tuleks pidada autoriteetseks allikaks. Olulise teabe puhul soovitatakse kasutada professionaalset inimtõlget. Me ei vastuta selle tõlke kasutamisest tulenevate arusaamatuste või valesti mõistmiste eest.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
